# Recommendation Engine

Inputs:
- User preferences: Collected from user settings and previous interactions.
- Historical data: Aggregated from past usage patterns and interactions.
- Location: Real-time location data from GPS and mobile devices.
- Current context: Contextual information such as time, weather, and local events.

Outputs:
- Recommendations for accommodations, activities, dining options, and travel routes: Suggestions tailored to user preferences and current location.

Usage:
- Assists users in planning their trips by suggesting personalized travel options based on their
preferences and current location.

---

For this recommendation engine, we'll use matrix factorization with ALS (Alternating Least Squares) from the implicit library. We'll simulate real-time location data and current context by adding randomness to the generated data.

This implementation showcases how to integrate user preferences, historical data, real-time location, and current context to provide personalized travel recommendations.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

Generating Sample Data

In [3]:
# Generate sample user preferences data
np.random.seed(42)
users = ['User1', 'User2', 'User3', 'User4', 'User5']
preferences = ['accommodation', 'activities', 'dining', 'travel_routes']

user_preferences = {
    'User': users,
    'Accommodation': np.random.randint(1, 6, size=5),
    'Activities': np.random.randint(1, 6, size=5),
    'Dining': np.random.randint(1, 6, size=5),
    'TravelRoutes': np.random.randint(1, 6, size=5),
}

# Generate sample historical data
items = ['PlaceA', 'PlaceB', 'PlaceC', 'PlaceD', 'PlaceE']
historical_data = {
    'User': np.random.choice(users, size=20),
    'Item': np.random.choice(items, size=20),
    'Rating': np.random.randint(1, 6, size=20),
    'Timestamp': pd.date_range(start='2023-01-01', periods=20, freq='D')
}

# Generate sample location data
locations = ['NY', 'LA', 'SF', 'TX', 'FL']
location_data = {
    'User': users,
    'CurrentLocation': np.random.choice(locations, size=5)
}

# Generate sample current context data
context_data = {
    'User': users,
    'Time': np.random.choice(['Morning', 'Afternoon', 'Evening', 'Night'], size=5),
    'Weather': np.random.choice(['Sunny', 'Rainy', 'Cloudy', 'Snowy'], size=5),
    'LocalEvents': np.random.choice(['Concert', 'Festival', 'Exhibition', 'None'], size=5)
}

# Convert to DataFrames
df_preferences = pd.DataFrame(user_preferences)
df_historical = pd.DataFrame(historical_data)
df_location = pd.DataFrame(location_data)
df_context = pd.DataFrame(context_data)

Model Building

In [4]:
# Create user-item interaction matrix
user_item_matrix = pd.pivot_table(df_historical, index='User', columns='Item', values='Rating', fill_value=0)

# Convert to sparse matrix format
sparse_user_item = csr_matrix(user_item_matrix.values)

# Initialize the ALS model
model = AlternatingLeastSquares(factors=10, regularization=0.1, iterations=20)

# Train the model
model.fit(sparse_user_item.T)

100%|██████████| 20/20 [00:00<?, ?it/s]


In [5]:
# Function to get top N recommendations for a given user
def get_top_n_recommendations(user, n=5):
    user_index = user_item_matrix.index.get_loc(user)
    recommendations = model.recommend(user_index, sparse_user_item[user_index], N=n)
    top_n_recommendations = [user_item_matrix.columns[i] for i in recommendations[0]]
    return top_n_recommendations

# Example usage
user = 'User1'
top_n_recommendations = get_top_n_recommendations(user)
print(f"Top recommendations for {user}: {top_n_recommendations}")

Top recommendations for User1: ['PlaceC', 'PlaceA', 'PlaceE', 'PlaceD', 'PlaceB']


In [6]:
# Function to adjust recommendations based on user preferences and context
def adjust_recommendations(user, recommendations):
    user_profile = df_preferences[df_preferences['User'] == user].iloc[0]
    context_profile = df_context[df_context['User'] == user].iloc[0]
    
    preference_scores = []
    for item in recommendations:
        score = 0
        score += user_profile['Accommodation'] * np.random.rand()
        score += user_profile['Activities'] * np.random.rand()
        score += user_profile['Dining'] * np.random.rand()
        score += user_profile['TravelRoutes'] * np.random.rand()
        
        if context_profile['Weather'] == 'Sunny':
            score += np.random.rand()
        if context_profile['LocalEvents'] != 'None':
            score += np.random.rand()
        
        preference_scores.append(score)
    
    adjusted_recommendations = sorted(zip(recommendations, preference_scores), key=lambda x: x[1], reverse=True)
    adjusted_recommendations = [rec[0] for rec in adjusted_recommendations]
    return adjusted_recommendations

# Adjusted recommendations based on user preferences and context
adjusted_recommendations = adjust_recommendations(user, top_n_recommendations)
print(f"Adjusted recommendations for {user}: {adjusted_recommendations}")

Adjusted recommendations for User1: ['PlaceE', 'PlaceD', 'PlaceC', 'PlaceA', 'PlaceB']
